In [1]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import torch
import transformers
import torch.nn.functional as F

In [2]:
model = RobertaForSequenceClassification.from_pretrained('roberta-large')
model.train()
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

In [3]:
??model.classifier.forward

In [4]:
??model.classifier

In [5]:
def classifer_output(self,features, **kwargs):
    print('features shape is: ', features.shape)
    x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
    print('x shape is: ', x.shape)
    x = self.dropout(x)
    x = self.dense(x)
    x = torch.tanh(x)
    x = self.dropout(x)
    x = self.out_proj(x)
    print(x)
    print(x.shape)
    print(x.size(0))
    return x
model.classifier.forward = classifer_output.__get__(model.classifier)
??model.classifier.forward

In [6]:
#testing what inputs are taken and how model takes them
inputs = tokenizer(["Hello, my dog is cute", "Today is a wonderful day", "I hate my boss"],\
                   padding = True, truncation = True, return_tensors="pt")
labels = torch.tensor([1, 1, 0]).unsqueeze(0)  # Batch size 1

# display(labels)
# display(labels.shape)
# display(inputs['input_ids'])
# display(inputs['input_ids'].shape)

outputs = model(input_ids = inputs['input_ids'], 
                attention_mask = inputs['attention_mask'],
                labels=labels
)
outputs

features shape is:  torch.Size([3, 8, 1024])
x shape is:  torch.Size([3, 1024])
tensor([[ 0.1313,  0.7318],
        [-0.0789,  0.7044],
        [ 0.1481,  0.6587]], grad_fn=<AddmmBackward>)
torch.Size([3, 2])
3


SequenceClassifierOutput(loss=tensor(0.5981, grad_fn=<NllLossBackward>), logits=tensor([[ 0.1313,  0.7318],
        [-0.0789,  0.7044],
        [ 0.1481,  0.6587]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [7]:
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [8]:
# load emotions dataset for fine-tuning model
from datasets import load_dataset
dataset = load_dataset("emotion")

Using custom data configuration default
Reusing dataset emotion (/home/advait/.cache/huggingface/datasets/emotion/default/0.0.0/aa34462255cd487d04be8387a2d572588f6ceee23f784f37365aa714afeb8fe6)


In [9]:
display(dataset)
train = dataset["train"]
test = dataset["test"]
validation = dataset["validation"]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [10]:
#tokenize dataset
train_encodings = tokenizer(train["text"], truncation=True, padding=True)
train.rename_column_("label", "labels")

validation_encodings = tokenizer(validation['text'], truncation = True, padding = True)
validation.rename_column_('label', 'labels')

display(train)
display(validation)

Dataset({
    features: ['text', 'labels'],
    num_rows: 16000
})

Dataset({
    features: ['text', 'labels'],
    num_rows: 2000
})

In [11]:
train.features

{'text': Value(dtype='string', id=None),
 'labels': ClassLabel(num_classes=6, names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], names_file=None, id=None)}

In [12]:
for item in train_encodings['input_ids']:
    if len(item) != 88:
        print('length is: ', len(item))
        raise ValueError

In [13]:
train_dataset = EmotionDataset(
    train_encodings, 
    train['labels'], 
)
validation_dataset = EmotionDataset(
    validation_encodings,
    validation['labels'],
)

In [14]:
#Change number of output labels
num_labels = 6
in_features = 1024

model.classifier = torch.nn.Linear(
    in_features = in_features,
    out_features = num_labels,
    bias = True
)
model.num_labels = num_labels
model.classifier

Linear(in_features=1024, out_features=6, bias=True)

In [15]:
??model.classifier.forward

In [16]:
def new_classifer_output(self,features, **kwargs):
    x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
    x = F.linear(x, self.weight, self.bias)
    return x

model.classifier.forward = new_classifer_output.__get__(model.classifier)
??model.classifier.forward

In [17]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle= True)

In [18]:
optim = transformers.AdamW(model.parameters(), lr = 5e-5)

In [19]:
# test_item = train_dataset.__getitem__(0)
# display(test_item['input_ids'].shape)
# display(test_item['attention_mask'].shape)
# display(test_item["labels"].unsqueeze(0).shape)
# outputs = model(
#     input_ids = test_item['input_ids'],
#     attention_mask = test_item['attention_mask'],
#     labels = test_item['labels'].unsqueeze(0)
# )
# outputs

In [20]:
for batch in train_loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels'].unsqueeze(0)
    print(input_ids.shape)
    print(attention_mask.shape)
    print(labels.shape)
    break

torch.Size([8, 88])
torch.Size([8, 88])
torch.Size([1, 8])


In [21]:
CUDA_LAUNCH_BLOCKING=1
torch.cuda.empty_cache()
model.to('cuda')
running_loss = 0.0
epoch_loss = 0.0
for epoch in range(3):
    for i, batch in enumerate(train_loader, 0):
        optim.zero_grad()
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].unsqueeze(0).to('cuda')
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        # print statistics
        running_loss += loss.item()
        epoch_loss += loss.item()
        if i % 500 == 499:    # print every 500 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 500))
            running_loss = 0.0

    print(f"\n epoch {epoch+1} loss is: ", epoch_loss/len(train_loader))
    epoch_loss = 0.0
print('Finished Training')
        

[1,   500] loss: 1.666
[1,  1000] loss: 1.618
[1,  1500] loss: 1.607
[1,  2000] loss: 1.595

 epoch 1 loss is:  1.621579302251339
[2,   500] loss: 1.607
[2,  1000] loss: 1.579
[2,  1500] loss: 1.593
[2,  2000] loss: 1.606

 epoch 2 loss is:  1.5961248015761376
[3,   500] loss: 1.594
[3,  1000] loss: 1.604
[3,  1500] loss: 1.578
[3,  2000] loss: 1.584

 epoch 3 loss is:  1.5902965607345103
Finished Training


In [30]:
test_inputs = tokenizer(["I am feeling very joyful today", "Today was a great day", "I love my life"],\
                       padding = True, truncation = True, return_tensors = "pt")
single_test = tokenizer(["Today was a great day"], return_tensors = "pt")
single_test

{'input_ids': tensor([[   0, 5625,   21,   10,  372,  183,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [31]:
model.to('cpu')
model(**single_test)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.5196,  0.6951, -0.8676, -0.0977, -0.0780, -1.4826]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [32]:
model(**test_inputs)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.5617,  0.7354, -0.6782,  0.0535,  0.0914, -1.4523],
        [ 0.5794,  0.8002, -0.8169, -0.0448,  0.1982, -1.4509],
        [ 0.5542,  0.7254, -0.8334, -0.0685,  0.0226, -1.4838]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [37]:
anger_input = tokenizer(["I had a horrible day"], return_tensors = "pt")
outputs = model(**anger_input)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.5727,  0.6854, -0.8902, -0.0637, -0.0450, -1.4432]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)